In [2]:
from bs4 import BeautifulSoup
import requests

In [4]:
import bs4

print(bs4.__version__)
print(requests.__version__)

4.12.3
2.31.0


# Empezamos el scraping

## 1) Obtener el HTML

In [4]:
URL_BASE = 'https://scrapepark.org/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

## 2) Parsear el HTML

In [5]:
soup = BeautifulSoup(html_obtenido,"html.parser")

## El método find() nos permite quedarnos con la información asociada a una etiqueta de HTML

In [ ]:
primer_h2 = soup.find('h2')
print(primer_h2)
print(primer_h2.text)

## El método find_all() busca todos los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase bs4.element.ResultSet).

In [ ]:
h2_todos=soup.find_all('h2')
print(h2_todos)

## Si usamos el parametro limit=1, emulamos al método find

In [ ]:
h2_uno_solo = soup.find_all('h2',limit=1)
print(h2_uno_solo)

## Podemos iterar sobre el objeto

In [ ]:
h2=soup.find_all('h2')
for i in h2:
    print(i.text)

## get_text() para más funcionalidades

In [ ]:
for seccion in h2:
    print(seccion.get_text(strip=True))

## Utilizando atributos de las etiquetas

## Clase

In [8]:
divs = soup.find_all('div', class_ = 'heading-container heading-center')
for div in divs:
    print(div)
    print(' ')

<div class="heading-container heading-center" id="about">
<h2>¿Por qué comprar con nosotros?</h2>
</div>
 
<div class="heading-container heading-center" id="products">
<h2>Nuestros productos</h2>
</div>
 
<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>
 
<div class="heading-container heading-center">
<h2>Testimonios de clientes</h2>
</div>
 


# Todas las etiquetas que tengan el atributo "src"

In [25]:
src_todos = soup.find_all('img')
for elemento in src_todos:
    print(elemento['alt'])
    '''if elemento['src'].endswith('.jpg'):
        print(elemento)'''

ScrapePark.org Logo
Parque de patinaje
Variedad de patinetas en nuestra tienda
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Cliente
Cliente
Cliente
Logo de ScrapePark.org
Logo de freeCodeCamp


# Bajar todas las imagenes

In [ ]:
for i,imagen in enumerate(src_todos):
    if imagen['src'].endswith('png'):
        print(imagen['src'])
        r=requests.get(f'https://scrapepark.org/spanish/{imagen['src']}')
        with open(f'imagen_{i}.png','wb') as f:
            f.write(r.content)            

# Información de tablas

In [27]:
URL_BASE = 'https://scrapepark.org/spanish'
URL_TABLA = soup.find_all('iframe')[0]['src']

request_tabla = requests.get(f'{URL_BASE}/{URL_TABLA}')

html_tabla = request_tabla.text
soup_tabla = BeautifulSoup(html_tabla,'html.parser')
soup_tabla.find('table')

productos_faltantes = soup_tabla.find_all(['th','td'], attrs={'style':'color: red;'})
productos_fal = [talle.text for talle in productos_faltantes ]

print(productos_fal)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [18]:
divs = soup.find_all('div', class_='detail-box')
productos = []
precios = []

for div in divs:
    if div.h6 is not None and 'Patineta' in div.h5.text:
        producto = div.h5.get_text(strip=True)
        precio = div.h6.get_text(strip=True).replace('$','')
        print(f'producto: {producto:<16} | precio: {precio}')
        productos.append(producto)
        precios.append(precio)       

producto: Patineta nueva1  | precio: 75
producto: Patineta usada2  | precio: 80
producto: Patineta nueva3  | precio: 68
producto: Patineta usada4  | precio: 70
producto: Patineta nueva5  | precio: 75
producto: Patineta nueva6  | precio: 58
producto: Patineta nueva7  | precio: 80
producto: Patineta nueva8  | precio: 35
producto: Patineta nueva9  | precio: 165
producto: Patineta usada10 | precio: 54
producto: Patineta usada11 | precio: 99
producto: Patineta nueva12 | precio: 110
Patineta nueva1 Patineta usada2 Patineta nueva3 Patineta usada4 Patineta nueva5 Patineta nueva6 Patineta nueva7 Patineta nueva8 Patineta nueva9 Patineta usada10 Patineta usada11 Patineta nueva12
75 80 68 70 75 58 80 35 165 54 99 110


# Cambios que dependen de la URL

In [31]:
URL_BASE = 'https://scrapepark.org/spanish/contact'

for i in range(1,3):
    URL_FINAL = f'{URL_BASE}{i}'
    print(URL_FINAL)
    r = requests.get(URL_FINAL)
    soup = BeautifulSoup(r.text,'html.parser')
    print(soup.h5.text) 

https://scrapepark.org/spanish/contact1
Texto que cambia entre páginas en contacto 1 :)
https://scrapepark.org/spanish/contact2
Texto que cambia entre páginas en contacto 2 :)


# Datos que no sabemos en que parte de la página se encuentran

In [7]:
# Expresiones regulares
import re

# 1. Obtener el HTML
URL_BASE = 'https://scrapepark.org/spanish'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2.'Parsear' ese HTML
soup = BeautifulSoup(html_obtenido,'html.parser')
telefonos = soup.find_all(string=re.compile('\d+-\d+-\d+'))
print(telefonos)

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jamir\AppData\Local\Temp\ipykernel_13648\229612336.py:11: SyntaxWarning: invalid escape sequence '\d'
  telefonos = soup.find_all(string=re.compile('\d+-\d+-\d+'))


[': 4-444-4444']


# Moviendonos por el arbol

In [52]:
copyrights = soup.find_all(string=re.compile('©'))
print(copyrights.text)


AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [9]:
primer_copyright=copyrights[0]
primer_copyright.parent

<p>© 2022 <span>Todos los derechos reservados</span>.
        <a href="https://html.design/" rel="noopener noreferrer" target="_blank">Creado con Free Html Templates</a>.
      </p>

# Otro ejemplo con elementos al mismo nivel

In [76]:
menu = soup.find(string=re.compile('MENÚ'))
print(menu.parent)
#menu.parent.find_next_siblings()

<h3 class="menu">MENÚ</h3>


# Comentario sobre exepciones

In [77]:
string_a_buscar = ['Descuentos','ofertas']

for string in string_a_buscar:
    try:
        resultado= soup.find(string=re.compile(string))
        print(resultado.text)
    except AttributeError :
        print(f'El string {string} no fue encontrado')           
        

Descuentos
Aprovechá nuestras ofertas.


# Almacenamiento de los datos

In [20]:
import csv

productos.insert(0,'productos')
precios.insert(0,'precios')
datos = dict(zip(productos,precios))
with open('datos.csv','w') as f:
    w= csv.writer(f)
    w.writerows(datos.items())

# Ejercicios

In [22]:
URL_='https://scrapepark.org/spanish/'
request=requests.get(URL_)
soup_= BeautifulSoup(request.text,'html.parser')    

### 1. Las patinetas que salgan menos que $68.

In [49]:
patinetas = soup_.find_all('div', class_='detail-box' )
for patineta in patinetas:
    if (patineta.h6 is not None) and ('Patineta' in patineta.h5.text):
        if int(patineta.h6.get_text(strip=True).replace('$','')) < 68 :
            print(patineta.h5.get_text(strip=True))

Patineta nueva6
Patineta nueva8
Patineta usada10


### 2. Las patinetas que en su nombre tengan un número mayor a 3.

In [69]:
for patineta in patinetas:
    if (patineta.h5 is not None) and ('Patineta' in patineta.h5.text) :
        if int(patineta.h5.text.strip().split(' ')[2]) > 3:
            print(patineta.h5.text.strip())        

Patineta usada 4
Patineta nueva 5
Patineta nueva 6
Patineta nueva 7
Patineta nueva 8
Patineta nueva 9
Patineta usada 10
Patineta usada 11
Patineta nueva 12


### 3. Traer cualquier texto de la pagina que tenga la palabra descuento u oferta.

In [80]:
for palabra in  ['Descuentos','ofertas'] :
    try:
        texto = soup_.find(string= re.compile(palabra))
        print(texto.text)
    except AttributeError:
        print(f'No se encontro la palabra {palabra} en el HTML')        

Descuentos
Aprovechá nuestras ofertas.


### 4. Generar un archivo .csv con dos columnas: Una conteniendo el nombre del cliente y otra su testimonio.

In [88]:
import pandas as pd
clientes=[]
comentarios=[]
clientes_ = soup_.find_all('div', class_='detail-box')
for cliente in clientes_:
    if cliente.h5 is not None and 'Cliente' in cliente.h5.text:
        clientes.append(cliente.h5.text.strip())
        comentarios.append(cliente.p.text.strip())

df = pd.DataFrame({
    'Cliente':clientes,
    'Comentario':comentarios
})

df.to_csv('clientes.csv')        

['Cliente 1', 'Cliente 2', 'Cliente 3']
['Los productos me encantaron y los precios son muy buenos. Lo recomiendo.', '¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.', 'Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.']
     Cliente                                         Comentario
0  Cliente 1  Los productos me encantaron y los precios son ...
1  Cliente 2  ¡La calidad y variedad de patinetas es impresi...
2  Cliente 3  Estoy muy conforme. Hay muchas patinetas y los...
